In [88]:
import MySQLdb

db = MySQLdb.connect(  # esto hace la conexion a la base de datos
    host="localhost",  # el puerto es 3308 porque es el puerto que
    user="root",  # le deje a mariaDB
    password="zamromxd",
    database="PokeSearch",
    port=3308,
)
c = db.cursor()

In [89]:
TYPES = dict()
c.execute(f"select * from tipe;")
result = c.fetchall()
for tipe in result:
 TYPES[tipe[0]] = tipe[1]
relations = dict()
for t_id in TYPES:
    if t_id not in relations:
        relations[t_id] = dict()
    c.execute(
        f"select relation,id_type_2 from relation_type where id_type_1={t_id} and relation like '%from' ;"
    )
    temp = c.fetchall()
    for rt in temp:
        if rt[0] not in relations[t_id]:
            relations[t_id][rt[0]] = []
            relations[t_id][rt[0]].append(rt[1])
        else:
            relations[t_id][rt[0]].append(rt[1])


def damages(T: list):
    D = dict()
    for t in T:
        for rt in relations[t]:
            for ti in relations[t][rt]:
                if ti in D:
                    if rt == "double_damage_from":
                        D[ti] *= 2
                    elif rt == "half_damage_from":
                        D[ti] *= 0.5
                    else:
                        D[ti] *= 0
                else:
                    if rt == "double_damage_from":
                        D[ti] = 2
                    elif rt == "half_damage_from":
                        D[ti] = 0.5
                    else:
                        D[ti] = 0
    return D



In [90]:
class Pokemon:
    def __init__(self, name: str):
        c.execute(f"select id,atk,satk,def,sdef,total_stat,name from pokemon where name='{name}';")
        data = c.fetchall()
        self.id = data[0][0]
        self.atk= data[0][1]
        self.defe= data[0][3]
        self.satk= data[0][2]
        self.sdef= data[0][4]
        self.total= data[0][5]
        self.name = data[0][6]
        c.execute(f"select id_type from pok_typ where id_pokemon={data[0][0]};")
        T = c.fetchall()
        self.types = []
        for t in T:
            self.types.append(t[0])
        self.damages = damages(self.types)
        
        self.moves = {"physical": dict(), "status": dict(), "special": dict()}
        c.execute(
            f"select M.name,M.id_type,M.damage_class from move as M inner join pok_mov as PM on  PM.id_move=M.id and PM.id_pokemon={self.id};"
        )
        M = c.fetchall()
        for att in M:
            if att[1] not in self.moves[att[2]]:
                self.moves[att[2]][att[1]] = list()
            self.moves[att[2]][att[1]].append(att[0])

In [91]:
def cantMove(principal:Pokemon):
    cant = dict()
    for clas in principal.moves:
        #c = 0
        cant[clas] = dict()
        for tipo in principal.moves[clas]:
            #c += len(principal.moves[clas][tipo])
            cant[clas][tipo] = len(principal.moves[clas][tipo])
        #cant[clas]['total'] = c
    #print(cant)
    return cant

In [136]:
p = "bulbasaur"
poke = Pokemon(p)
# p = input('Nombre del pokemon: ')
# c.execute(f"select * from pokemon where name='{p}';")
# result = c.fetchall()
pokes = dict()
c.execute(f"select name from pokemon where total_stat>={poke.total-20} and total_stat<={poke.total+20} and id!={poke.id};")
#c.execute(f'select name from pokemon;')
result = c.fetchall()
for r in result:
 pokes[r[0]] = Pokemon(r[0])
 #print(r[0])

In [128]:
def calcular(main:Pokemon, other: Pokemon):
    ptM = 0
    ptO = 0
    if main.atk > main.satk:
        pri_atk = "physical"
    elif main.atk < main.satk:
        pri_atk = "special"
    else:
        pri_atk = False
    if main.defe > main.sdef:
        pri_def = "physical"
    elif main.defe < main.sdef:
        pri_def = "special"
    else:
        pri_def = False
    if other.atk > other.satk:
        other_atk = "physical"
    elif other.atk < other.satk:
        other_atk = "special"
    else:
        other_atk = False
    if other.defe > other.sdef:
        other_def = "physical"
    elif other.defe < other.sdef:
        other_def = "special"
    else:
        other_def = False

    # print(f'Principal: {main.name}\n\tataque: {pri_atk}\n\tdefensa: {pri_def}')
    # print(f'\tTipos:')
    # for typ in main.types:
    # print(f'\t\t{TYPES[typ]}')
    priCant = cantMove(main)
    for classM in priCant:
        # p = priCant[classM]['total']
        for t in priCant[classM]:
            P = ''
            p = priCant[classM][t]
            pp = p
            P += str(p)
            if pri_atk and classM==pri_atk:
                p += pp/4
                # p *= 1.25
                P += '*1.25'
            if other_def and other_def==classM:
                p -= pp/4
                # p /= 1.25
                P += "/1.25"
            if t in main.types:
                p += pp / 2
                # p *= 1.5
                P += "*1.5"
            if t in other.damages:
                p *= other.damages[t]
                P += f"*{other.damages[t]}"

            # print(f'{classM}: {TYPES[t]}({t}): {p}  - {P}')
            ptM +=p
    # print(f'Puntos: {ptM}')

    # print()
    # print(f"Otro: {other.name}\n\tataque: {other_atk}\n\tdefensa: {other_def}")
    # print(f"\tTipos:")
    # for typ in other.types:
    # print(f"\t\t{TYPES[typ]}")
    otherCant = cantMove(other)
    for classM in otherCant:
        # p = priCant[classM]['total']
        for t in otherCant[classM]:
            P = ""
            p = otherCant[classM][t]
            pp = p 
            P += str(p)
            if other_atk and classM == other_atk:
                p += pp / 4
                # p *= 1.25
                P += "*1.25"

            if pri_def and pri_def == classM:
                p = pp / 4
                # p /= 1.25
                P += "/1.25"
            if t in other.types:
                p += pp / 2
                # p *= 1.5
                P += "*1.5"
            if t in main.damages:
                p *= main.damages[t]
                P += f"*{main.damages[t]}"

            # print(f"{classM}: {TYPES[t]}({t}): {p}  - {P}")
            ptO += p
    # print(f"Puntos: {ptO}")
    return ptM-ptO + (main.total-other.total)
    #return ptM - ptO 

In [138]:
calcular(poke, pokes["pikachu"])

9.5

In [137]:
PL = list()
for pp in pokes:
    PL.append((pp, calcular(poke, pokes[pp])))
    PL.sort(key=lambda x: x[1])
print(f"{p}:")
print(f"\tBetter: {PL[0:3]}")
print(f"\tWorst: {PL[-3:]}")

bulbasaur:
	Better: [('drowzee', -66.125), ('delibird', -64.375), ('natu', -58.9375)]
	Worst: [('binacle', 105.8125), ('sobble', 108.25), ('larvitar', 118.25)]


In [133]:
for p in pokes:
    poke = Pokemon(p)
    PL = list()
    for pp in pokes:
        PL.append((pp, calcular(poke, pokes[pp])))
    # print(PL)
    PL.sort(key=lambda x: x[1])
    #print(f'{p}\t{PL}')
    print(f'{p}:')
    print(f'\tBetter: {PL[0:3]}')
    print(f'\tWorst: {PL[-3:]}')

charmander:
	Better: [('geodude', -48.75), ('binacle', -46.0625), ('frillish', -44.625)]
	Worst: [('snorunt', 118.875), ('snover', 126.5), ('sewaddle', 141.125)]
squirtle:
	Better: [('frillish', -50.875), ('pumpkaboo-average', -46.125), ('clefairy', -37.125)]
	Worst: [('numel', 166.625), ('geodude', 180.0), ('larvitar', 198.875)]
pikachu:
	Better: [('golett', -48.875), ('pumpkaboo-average', -43.25), ('phantump', -39.375)]
	Worst: [('remoraid', 123.25), ('sobble', 127.125), ('ducklett', 180.125)]
sandshrew:
	Better: [('frillish', -84.5), ('pumpkaboo-average', -64.375), ('clefairy', -51.25)]
	Worst: [('beldum', 98.0), ('sizzlipede', 99.0), ('salandit', 110.4375)]
clefairy:
	Better: [('honedge', 8.0625), ('pumpkaboo-average', 17.5), ('stunky', 19.875)]
	Worst: [('croagunk', 168.9375), ('deino', 170.5), ('clobbopus', 180.625)]
vulpix:
	Better: [('teddiursa', -74.5), ('geodude', -69.875), ('binacle', -68.8125)]
	Worst: [('grubbin', 77.125), ('beldum', 93.375), ('sewaddle', 102.875)]
oddish: